## 1. Setup & Configuration

In [52]:
import json
import os
import sys
import random
from typing import List, Dict

# Setup path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

print("✅ Setup hoàn tất!")

✅ Setup hoàn tất!


In [53]:
# =============================================================================
# CẤU HÌNH - THAY ĐỔI Ở ĐÂY
# =============================================================================

# Đường dẫn Input (thư mục chứa các folder paper với labels.json)
INPUT_DIR = os.path.abspath('../../data_output_v2')

# Đường dẫn Output (thư mục xuất dataset)
OUTPUT_DIR = os.path.abspath('../../dataset_final')

# Prefix năm-tháng của papers (VD: 2403 = March 2024)
YYMM_PREFIX = '2403'

# Giới hạn số lượng papers (None = lấy tất cả)
LIMIT = 780  # Đặt None để lấy full dataset

# Hoặc dùng Range (ưu tiên hơn LIMIT nếu cả 2 đều set)
USE_RANGE = False
RANGE_START = 0
RANGE_END = 100

# Random seed
RANDOM_SEED = 42

# =============================================================================
print(f"📂 Input:  {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"📅 Prefix: {YYMM_PREFIX}")
print(f"🔢 Limit:  {LIMIT}")

📂 Input:  d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_output_v2
📂 Output: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\dataset_final
📅 Prefix: 2403
🔢 Limit:  780


## 2. Helper Functions

In [54]:
import re

def check_bibtex_format(input_dir: str, folder_name: str) -> bool:
    """
    Kiểm tra xem paper có labels.json với TẤT CẢ content là BibTeX chuẩn không.
    
    Kiểm tra trường 'content' trong labels.json:
    - True nếu TẤT CẢ items có content bắt đầu với @article, @misc, etc.
    - False nếu có BẤT KỲ item nào là flat text (\newblock) hoặc không có file
    """
    labels_path = os.path.join(input_dir, folder_name, 'labels.json')
    
    if not os.path.exists(labels_path):
        return False
    
    try:
        with open(labels_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if not isinstance(data, list) or len(data) == 0:
            return False
        
        # Kiểm tra TẤT CẢ items phải có content bắt đầu với @
        bibtex_pattern = re.compile(r'^\s*@\w+\s*\{', re.IGNORECASE)
        
        for item in data:
            content = item.get('content', '')
            if not content or not bibtex_pattern.match(content):
                # Có ít nhất 1 item không phải BibTeX chuẩn -> skip paper này
                return False
        
        return True
        
    except Exception:
        return False


def load_selected_papers(input_dir: str, folder_names: List[str]) -> Dict[str, List[dict]]:
    """
    Đọc dữ liệu từ danh sách folder.
    
    Returns:
        Dict {paper_id: [list of references]}
    """
    papers_map = {}
    print(f"🔄 Loading data from {len(folder_names)} folders...")

    for folder_name in folder_names:
        file_path = os.path.join(input_dir, folder_name, 'labels.json')
        if not os.path.exists(file_path):
            continue
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            if isinstance(data, list) and len(data) > 0:
                papers_map[folder_name] = data
        except Exception as e:
            print(f"   ⚠️ Error loading {folder_name}: {e}")
            
    print(f"   ✅ Loaded {len(papers_map)} papers with valid labels")
    return papers_map


def save_json(data_list: List, filepath: str):
    """Lưu list ra file JSON."""
    if not data_list:
        return
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)
    print(f"   💾 Saved: {os.path.basename(filepath)} ({len(data_list)} items)")


print("✅ Helper functions defined!")

✅ Helper functions defined!


## 3. Scan & Filter Folders

In [55]:
# Kiểm tra input directory
if not os.path.exists(INPUT_DIR):
    raise FileNotFoundError(f"❌ Input directory not found: {INPUT_DIR}")

# Quét tất cả folders
all_items = os.listdir(INPUT_DIR)
all_folders = [
    name for name in all_items 
    if os.path.isdir(os.path.join(INPUT_DIR, name)) 
    and name.startswith(YYMM_PREFIX)
]
all_folders.sort()

if not all_folders:
    raise ValueError(f"❌ No folders found with prefix '{YYMM_PREFIX}'")

print(f"📋 Found {len(all_folders)} folders matching prefix '{YYMM_PREFIX}'")

# --- LỌC CHỈ LẤY PAPERS CÓ TẤT CẢ REFS LÀ BIBTEX FORMAT CHUẨN ---
print(f"\n🔍 Checking labels.json content format...")
print(f"   (Only papers where ALL refs start with @article/@misc/...)")
valid_folders = []
skipped_folders = []

for folder in all_folders:
    if check_bibtex_format(INPUT_DIR, folder):
        valid_folders.append(folder)
    else:
        skipped_folders.append(folder)

print(f"\n   ✅ Valid (all BibTeX format): {len(valid_folders)} papers")
print(f"   ⏭️  Skipped (has flat text refs): {len(skipped_folders)} papers")

if skipped_folders[:5]:
    print(f"\n   Sample skipped: {skipped_folders[:5]}")

if not valid_folders:
    raise ValueError(f"❌ No papers with valid BibTeX format found!")

print(f"\n   First valid: {valid_folders[0]}")
print(f"   Last valid:  {valid_folders[-1]}")

📋 Found 5000 folders matching prefix '2403'

🔍 Checking labels.json content format...
   (Only papers where ALL refs start with @article/@misc/...)

   ✅ Valid (all BibTeX format): 762 papers
   ⏭️  Skipped (has flat text refs): 4238 papers

   Sample skipped: ['2403-00532', '2403-00534', '2403-00535', '2403-00536', '2403-00537']

   First valid: 2403-00530
   Last valid:  2403-05519


In [56]:
# Áp dụng Limit hoặc Range
random.seed(RANDOM_SEED)

if USE_RANGE:
    target_folders = valid_folders[RANGE_START:RANGE_END]
    print(f"✂️  Mode: RANGE [{RANGE_START} -> {RANGE_END}]")
elif LIMIT and LIMIT < len(valid_folders):
    temp_list = list(valid_folders)
    random.shuffle(temp_list)
    target_folders = temp_list[:LIMIT]
    print(f"🎲 Mode: RANDOM LIMIT ({LIMIT} papers)")
else:
    target_folders = valid_folders
    print(f"🚀 Mode: FULL DATASET")

print(f"   👉 Will process {len(target_folders)} papers")

🚀 Mode: FULL DATASET
   👉 Will process 762 papers


## 4. Load Data

In [57]:
# Load papers
papers_db = load_selected_papers(INPUT_DIR, target_folders)
all_loaded_ids = list(papers_db.keys())

if not all_loaded_ids:
    raise ValueError("❌ No valid labels.json found in selected folders!")

# Thống kê
total_refs = sum(len(refs) for refs in papers_db.values())
print(f"\n📊 Statistics:")
print(f"   Papers loaded: {len(papers_db)}")
print(f"   Total refs:    {total_refs}")
print(f"   Avg refs/paper: {total_refs/len(papers_db):.1f}")

🔄 Loading data from 762 folders...
   ✅ Loaded 762 papers with valid labels

📊 Statistics:
   Papers loaded: 762
   Total refs:    16489
   Avg refs/paper: 21.6


## 5. Select Manual Papers

In [58]:
# Tìm papers có >= 20 refs cho Manual
MIN_REFS_FOR_MANUAL = 20

candidates_manual = []
for pid, refs in papers_db.items():
    if len(refs) >= MIN_REFS_FOR_MANUAL:
        candidates_manual.append((pid, len(refs)))

candidates_manual.sort(key=lambda x: x[1], reverse=True)

print(f"🎯 Candidates for Manual (>= {MIN_REFS_FOR_MANUAL} refs): {len(candidates_manual)} papers")
print("\n   Top 10:")
for pid, count in candidates_manual[:10]:
    print(f"   - {pid}: {count} refs")

🎯 Candidates for Manual (>= 20 refs): 306 papers

   Top 10:
   - 2403-04279: 214 refs
   - 2403-05131: 185 refs
   - 2403-01313: 180 refs
   - 2403-02619: 180 refs
   - 2403-01202: 148 refs
   - 2403-00533: 132 refs
   - 2403-02469: 126 refs
   - 2403-05175: 119 refs
   - 2403-05030: 100 refs
   - 2403-02203: 99 refs


In [59]:
# Kiểm tra đủ candidates
if len(candidates_manual) < 5:
    print("❌ ERROR: Not enough candidates for manual selection!")
    print(f"   Required: 5 papers with >= {MIN_REFS_FOR_MANUAL} refs")
    print(f"   Found: {len(candidates_manual)}")
    print("   💡 Suggestion: Increase LIMIT or use full dataset")
else:
    print("✅ Đủ candidates cho Manual selection!")

✅ Đủ candidates cho Manual selection!


In [60]:
# Chọn 5 bài Manual (3 Train, 1 Val, 1 Test)
random.seed(RANDOM_SEED)
candidate_ids = [x[0] for x in candidates_manual]
random.shuffle(candidate_ids)

selected_manual = candidate_ids[:5]

manual_train_ids = selected_manual[:3]
manual_val_ids = selected_manual[3:4]
manual_test_ids = selected_manual[4:5]

print("✍️  SELECTED MANUAL PAPERS:")
print(f"   Train (3): {manual_train_ids}")
print(f"   Val   (1): {manual_val_ids}")
print(f"   Test  (1): {manual_test_ids}")

✍️  SELECTED MANUAL PAPERS:
   Train (3): ['2403-04225', '2403-03951', '2403-01980']
   Val   (1): ['2403-04970']
   Test  (1): ['2403-00803']


## 6. Split Auto Papers

In [61]:
# Auto = (Tất cả) - (5 bài manual)
auto_pool_ids = [pid for pid in all_loaded_ids if pid not in selected_manual]

random.shuffle(auto_pool_ids)
total_auto = len(auto_pool_ids)

# Chia 80/10/10
tr_end = int(total_auto * 0.8)
val_end = int(total_auto * 0.9)

if total_auto < 10:
    print("⚠️ Small auto dataset. Putting all into Train.")
    auto_train_ids = auto_pool_ids
    auto_val_ids = []
    auto_test_ids = []
else:
    auto_train_ids = auto_pool_ids[:tr_end]
    auto_val_ids = auto_pool_ids[tr_end:val_end]
    auto_test_ids = auto_pool_ids[val_end:]

print(f"\n🤖 AUTO DATASET SPLIT ({total_auto} papers):")
print(f"   Train: {len(auto_train_ids)} papers")
print(f"   Val:   {len(auto_val_ids)} papers")
print(f"   Test:  {len(auto_test_ids)} papers")


🤖 AUTO DATASET SPLIT (757 papers):
   Train: 605 papers
   Val:   76 papers
   Test:  76 papers


## 7. Save Dataset

In [62]:
def process_and_save(subset_name: str, auto_ids: List[str], manual_ids: List[str]):
    """Gộp data và lưu file."""
    # Gộp Auto data
    auto_data = []
    for pid in auto_ids:
        auto_data.extend(papers_db[pid])
    
    # Gộp Manual data
    manual_data = []
    for pid in manual_ids:
        manual_data.extend(papers_db[pid])
        
    base_path = os.path.join(OUTPUT_DIR, subset_name)
    print(f"\n📂 Writing {subset_name.upper()}...")
    
    if auto_data:
        save_json(auto_data, os.path.join(base_path, 'auto.json'))
    if manual_data:
        save_json(manual_data, os.path.join(base_path, 'manual.json'))
    
    return len(auto_data), len(manual_data)

In [63]:
def count_before_process(auto_ids: List[str]):
    d = []
    for pid in auto_ids:
        d.extend(papers_db[pid])
    return len(d)

In [64]:
auto_val_id = 0
auto_test_id = 0

for id in range(len(auto_val_ids)):
    if count_before_process(auto_val_ids[id:id + 1]) >= 20:
        auto_val_id = id


for id in range(len(auto_test_ids)):
    if count_before_process(auto_test_ids[id:id + 1]) >= 20:
        auto_test_id = id

In [65]:
# Tạo output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Lưu từng partition
train_auto, train_manual = process_and_save('train', auto_train_ids, manual_train_ids)

# chỉ lấy 1 tệp
val_auto, val_manual = process_and_save('validation', auto_val_ids[auto_val_id:auto_val_id + 1], manual_val_ids)
test_auto, test_manual = process_and_save('test', auto_test_ids[auto_test_id:auto_test_id + 1], manual_test_ids)

print(f"\n🎉 DONE! Dataset saved to: {OUTPUT_DIR}")


📂 Writing TRAIN...
   💾 Saved: auto.json (13222 items)
   💾 Saved: manual.json (126 items)

📂 Writing VALIDATION...
   💾 Saved: auto.json (35 items)
   💾 Saved: manual.json (72 items)

📂 Writing TEST...
   💾 Saved: auto.json (63 items)
   💾 Saved: manual.json (22 items)

🎉 DONE! Dataset saved to: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\dataset_final


## 8. Summary

In [66]:
# Tổng kết
print("="*60)
print("📊 DATASET SPLIT SUMMARY")
print("="*60)
print(f"\n{'Partition':<12} {'Auto Refs':<12} {'Manual Refs':<12} {'Total':<10}")
print("-"*50)
print(f"{'Train':<12} {train_auto:<12} {train_manual:<12} {train_auto+train_manual:<10}")
print(f"{'Validation':<12} {val_auto:<12} {val_manual:<12} {val_auto+val_manual:<10}")
print(f"{'Test':<12} {test_auto:<12} {test_manual:<12} {test_auto+test_manual:<10}")
print("-"*50)
total = train_auto + train_manual + val_auto + val_manual + test_auto + test_manual
print(f"{'TOTAL':<12} {train_auto+val_auto+test_auto:<12} {train_manual+val_manual+test_manual:<12} {total:<10}")
print("="*60)

📊 DATASET SPLIT SUMMARY

Partition    Auto Refs    Manual Refs  Total     
--------------------------------------------------
Train        13222        126          13348     
Validation   35           72           107       
Test         63           22           85        
--------------------------------------------------
TOTAL        13320        220          13540     


In [67]:
# Verify output files
print("\n📁 Output Files:")
for partition in ['train', 'validation', 'test']:
    partition_path = os.path.join(OUTPUT_DIR, partition)
    if os.path.exists(partition_path):
        files = os.listdir(partition_path)
        print(f"\n   {partition}/")
        for f in files:
            fpath = os.path.join(partition_path, f)
            size = os.path.getsize(fpath)
            print(f"      └── {f}: {size:,} bytes")


📁 Output Files:

   train/
      └── auto.json: 16,205,452 bytes
      └── labels.json: 16,324,211 bytes
      └── manual.json: 118,762 bytes

   validation/
      └── auto.json: 32,561 bytes
      └── labels.json: 96,074 bytes
      └── manual.json: 96,074 bytes

   test/
      └── auto.json: 58,352 bytes
      └── labels.json: 21,747 bytes
      └── manual.json: 21,747 bytes


---
## Next Steps

Sau khi chạy notebook này, bạn có thể:

1. **Notebook 01:** `01_completeness_check_data_cleaning.ipynb` - Kiểm tra và làm sạch dữ liệu
2. **Notebook 02:** `02_data_augmentation.ipynb` - Sinh negative samples
3. **Notebook 03:** `03_feature_engineering.ipynb` - Trích xuất features
4. **Notebook 04:** `04_modeling.ipynb` - Huấn luyện model
5. **Notebook 05:** `05_evaluation_ranking.ipynb` - Đánh giá và ranking
